In [1]:
import  cv2
import numpy as np
from keras.models import load_model, Sequential
from keras.layers import Dense, BatchNormalization, Dropout

from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

#For local CPU usage:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

#face detection and extraction
from mtcnn.mtcnn import MTCNN
from PIL import Image
from numpy import asarray

Using TensorFlow backend.


In [2]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

EMOTION_DICT = {1:"ANGRY", 2:"DISGUST", 3:"FEAR", 4:"HAPPY", 5:"NEUTRAL", 6:"SAD", 7:"SURPRISE"}

### Define functions and model from training, necessary for prediction

In [3]:
detector = MTCNN()

def detect_faces(image):
    faces = detector.detect_faces(image)
    return np.array(faces)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [4]:
def extract_face_from_image(image, required_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):
    face = detect_faces(image) # content of face is a python dict

    
    print(len(face))
    if len(face) == 0:
        return []
    else:
        # extract the bounding box from the requested face
        box = np.asarray(face[0]['box'])
        box[box < 0] = 0
        x1, y1, width, height =  box

        x2, y2 = x1 + width, y1 + height
        # extract the face
        face_boundary = image[y1:y2, x1:x2]

        # resize pixels to the model size
        face_image = Image.fromarray(face_boundary)
        face_image = face_image.resize(required_size)
        face_array = asarray(face_image)

        return face_array

In [5]:
def get_bounding_box(image, required_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):
    face = detect_faces(image) # content of face is a python dict
    
    if len(face) == 0:
        return []
    elif len(face) == 1:
        # extract the bounding box from the requested face
        box = np.asarray(face[0]['box'])
        box[box < 0] = 0
        return np.array(box)
    else:
        box = []
        for elem in face:
            b = np.asarray(elem['box'])
            b[b < 0] = 0
            box.append(b)
        return np.array(box)

In [6]:
model_VGGFace = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')    
    
def get_face_embedding(face):
    samples = asarray(face, 'float32')
    
    # prepare the data for the model
    samples = preprocess_input(samples, version=2)
        
    return model_VGGFace.predict(samples)

In [7]:
def model_top(input_shape):
    model = Sequential()
        
    model.add(Dense(512, activation='relu', input_dim = input_shape))
    model.add(Dropout(0.1))
    
    model.add(Dense(256, activation='relu'))
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim = 2, activation='tanh')) 
    
    return model

In [8]:
model_top = model_top(2048)
model_top.load_weights("model_top.h5")

c:\users\tobias\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=2)`
  del sys.path[0]


### Create the application

In [9]:
def return_prediction(path):
    #converting image to RGB color and save it
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    
    #detect face in image, crop it then resize it
    face = extract_face_from_image(img)
    
    if face.ndim == 3:
        face = face.reshape((1, face.shape[0], face.shape[1], face.shape[2]))
    
    #get embedding for face
    VGG_pred = get_face_embedding(face)
    
    #make prediction and display the result
    top_pred = model_top.predict(VGG_pred)
    
    val = int(top_pred[0][0] * 10)
    ar = int(top_pred[0][1] * 10)
    print("result: " + str(val) + ", " + str(ar))
    return val, ar

In [10]:
## ## face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

cap = cv2.VideoCapture(0)

def run_app(text, cap):
    while(True):
        ret, img = cap.read()
        img_color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(text), (95,30), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(img, "Press SPACE: FOR EMOTION", (5,470), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(img, "Hold Q: To Quit", (460,470), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        
        
        box = get_bounding_box(img_color)
        if box == []:
            print("No bounding box")
        elif box.ndim == 1:
            x,y,w,h = box
            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
            cv2.imshow("Emotion Recognition - MasterThesis", img)
        elif box.ndim >= 2:
            for elem in box:
                x, y, w, h = elem
                cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
            cv2.imshow("Emotion Recognition - MasterThesis", img)
        
        if cv2.waitKey(1) == ord(' '):
            cv2.imwrite("test.jpg", img)
            valence, arousal = return_prediction("test.jpg")
            run_app("Valence: " + str(int(valence)) + ", Arousal: " + str(int(arousal)) , cap)
            # run_app("No Emotion", cap)
            break
            
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

In [11]:
run_app("None", cap)

c:\users\tobias\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


1
result: -9, 10
No bounding box
No bounding box
1
result: -9, 10
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
No bounding box
